In [2]:
# Sifting

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from random import randrange

In [4]:
def print_outcomes_in_reserve(counts):
    for outcome in counts:
        reverse_outcome = ''
        for i in outcome:
            reverse_outcome = i + reverse_outcome
        return reverse_outcome
    


def SendState(qc1,qc2,qc1_name):
    """
    This function takes the output of the qc1 and 
    initializes another circuit qc2 with the same state
    """

    #Quantum state retrieved from qc1
    qs = qc1.qasm().split(sep = ';')[4:-1]


    #Process the code to get the instructions

    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

    
    #Parsing the instruction and applying to new circuit
        
    for instruction in qs:
        if instruction[0] == 'x':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])

            else:
                old_qr = int(instruction[5:-1])
            qc2.x(qreg[old_qr])

        elif instruction[0] == 'h':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])

            qc2.h(qreg[old_qr])


        elif instruction[0] == 'm':
            pass
        else:
            raise Exception('Unable to parse instruction')
        


qreg = QuantumRegister(16)
creg = ClassicalRegister(16)

asja = QuantumCircuit(qreg, creg, name = 'Asja')

send = []
asja_basis = []
balvis_basis = []


#Creating random bit string
for i in range(16):
    bit = randrange(2)
    send.append(bit)


#Preparing qubits, apply X gate if bit is 1

for i, n in enumerate(send):
    if n == 1:
        asja.x(qreg[i])


#Encoding
        
for i in range(16):
    r = randrange(2) #Asja randomly picks a basis
    if r == 0: #if bit is 0 then she encoded in Z basis
        asja_basis.append('Z')
    else:
        asja.h(qreg[i])
        asja_basis.append('X')

balvis = QuantumCircuit(qreg, creg, name = 'Balvis')


SendState(asja, balvis, 'Asja') #Asja sends states to Balvis

#Balvis measures qubits
for i in range(16):
    r = randrange(2)
    if r==0: #if bit p, then measures in Z basis
        balvis.measure(qreg[i],creg[i])
        balvis_basis.append('Z')

    else: #if bit 1, then measures in X basis
        balvis.h(qreg[i])
        balvis.measure(qreg[i],creg[i])
        balvis_basis.append('X')


#Running the circuit
        
job = execute(balvis, Aer.get_backend('qasm_simulator'),shots = 1)
counts = job.result().get_counts(balvis)
counts = print_outcomes_in_reserve(counts)


#Saving Balvis received string as a list
received = list(map(int, counts))

print("Asja sent:", send)
print("Asja encoding basis:", asja_basis)
print("Balvis received:", received)
print("Balvis decoding basis:", balvis_basis)

Asja sent: [1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1]
Asja encoding basis: ['X', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z']
Balvis received: [1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
Balvis decoding basis: ['X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X']


C:\Users\jrnat\AppData\Local\Temp\ipykernel_28180\3920497357.py:17: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.qasm()`` is deprecated as of qiskit 0.46. It will be removed in the 1.0.0 release. Instead use the qiskit.qasm2.dump() or qiskit.qasm2.dumps() function
  qs = qc1.qasm().split(sep = ';')[4:-1]
C:\Users\jrnat\AppData\Local\Temp\ipykernel_28180\3920497357.py:106: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(balvis, Aer.get_backend('qasm_simulator'),shots = 1)


In [6]:
#Now the additional code for sifting

#Sifting
asja_key=[] #Asjas register for matching rounds
balvis_key=[] #Balvis register for matching rounds
for j in range(0,len(asja_basis)): #Going through list of bases 
    if asja_basis[j] == balvis_basis[j]: #Comparing
        asja_key.append(send[j])
        balvis_key.append(received[j]) #Keeping key bit if bases matched
    else:
        pass #Discard round if bases mismatched

print("Asjas key =", asja_key)
print("Balvis key =", balvis_key)

Asjas key = [1, 0, 0, 0, 0, 0, 1, 1, 0, 0]
Balvis key = [1, 0, 0, 0, 0, 0, 1, 1, 0, 0]


In [ ]:
#So, we got the key strings for which the same bases was used for encoding and decoding

In [7]:
#Now let us find the QBER for the circuit 1

rounds = len(asja_key)//3    #To divide without remainer, use //
errors=0
for i in range(rounds):
    bit_index = randrange(len(asja_key)) 
    tested_bit = asja_key[bit_index]
    print ("Asja randomly selected bit index =", bit_index, ", and its value is = ", tested_bit)
    if asja_key[bit_index]!=balvis_key[bit_index]: #comparing tested rounds
        errors=errors+1 #calculating errors
    #removing tested bits from key strings
    del asja_key[bit_index] #Use del to specify the index of the element you want to delete
    del balvis_key[bit_index]
QBER=errors/rounds #calculating QBER
        
print("QBER value =", QBER)
print("Asja's secret key =", asja_key)
print("Balvis' secret key =", balvis_key)

Asja randomly seleccted bit index =  5 , and its value is:  0
QBER value =  0.0
Asja's secret key=  [1, 0, 0, 0, 0, 0, 1, 1, 0, 0]
Balvis's secret key =  [1, 0, 0, 0, 0, 0, 1, 1, 0, 0]
Asja randomly seleccted bit index =  8 , and its value is:  0
QBER value =  0.0
Asja's secret key=  [1, 0, 0, 0, 0, 0, 1, 1, 0, 0]
Balvis's secret key =  [1, 0, 0, 0, 0, 0, 1, 1, 0, 0]
Asja randomly seleccted bit index =  0 , and its value is:  1
QBER value =  0.0
Asja's secret key=  [1, 0, 0, 0, 0, 0, 1, 1, 0, 0]
Balvis's secret key =  [1, 0, 0, 0, 0, 0, 1, 1, 0, 0]
